# Temperature Interpolation
## Tzu-Yu Ma
### April 11, 2024

In [1]:
import arcpy
import os
import psycopg2
import random

#### Workspace

In [ ]:
arcpy.env.workspace = r"D:\spring2024\GIS5572\Lab3\Lab3\Lab3.gdb"

#### Connect to PostGIS database

In [67]:
conn = psycopg2.connect(
    dbname="gis5572",
    user="postgres",
    password="",
    host="",
    port="5432"
)
cur = conn.cursor()

#### Prepare temperature data

In [35]:
cur.execute("SELECT * FROM mn_temperature")

# Fetch the results
results = cur.fetchall()

# Print the results
print(len(results))
# Close the cursor and connection
cur.close()
conn.close()

4637


#### Filter data

In [5]:
# select date 2023-03-20, becuase station MN006 has data since 2023-03-20
arcpy.management.SelectLayerByAttribute('mn_temperature', 'NEW_SELECTION',"date = date '2023-03-20'")
arcpy.management.CopyFeatures('mn_temperature', 'mn_temp_0320')

<Result 'D:\\spring2024\\GIS5572\\Lab3\\Lab3\\Lab3.gdb\\mn_temp_0320'>

#### Interpolation algorithms results

In [7]:
# ORDINARY_KRIGING
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="mn_temp_0320",
        z_field="max_tmpf",
        kriging_model="Spherical # # # #",
        cell_size=0.0218500823959998,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(arcpy.env.workspace + "\\Kriging_ord_temp")

In [9]:
# UNIVERSAL_KRIGING
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="mn_temp_0320",
        z_field="max_tmpf",
        kriging_model="LinearDrift 0.021850 # # #",
        cell_size=0.0218500823959998,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(arcpy.env.workspace + "\\Kriging_uni_temp")

In [10]:
# IDW
with arcpy.EnvManager(scratchWorkspace=arcpy.env.workspace):
    out_raster = arcpy.sa.Idw(
        in_point_features="mn_temp_0320",
        z_field="max_tmpf",
        cell_size=0.0218500823959998,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(arcpy.env.workspace + "\\Idw_temp")

#### Saves the interpolate results to PostGIS

In [2]:
def raster_pts_SDE(input_raster, output_pt_name):
    # convert raster to point
    arcpy.conversion.RasterToPoint(
        in_raster=input_raster,
        out_point_features=arcpy.env.workspace + "\\" + output_pt_name,
        raster_field="Value")
    # save data to PostGIS    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features=arcpy.env.workspace + "\\" + output_pt_name,
        Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde")
    
    print(f"{input_raster} has been converted to points and stored into SDE as {output_pt_name}.")


In [3]:
# ORDINARY_KRIGING
raster_pts_SDE ('Kriging_ord_temp', 'Krig_ord_pts_temp')

# UNIVERSAL_KRIGING
raster_pts_SDE ('Kriging_uni_temp', 'Krig_uni_pts_temp')

# IDW
raster_pts_SDE ('Idw_temp', 'Idw_pts_temp')

Kriging_ord_temp has been converted to points and stored into SDE as Krig_ord_pts_temp.
Kriging_uni_temp has been converted to points and stored into SDE as Krig_uni_pts_temp.
Idw_temp has been converted to points and stored into SDE as Idw_pts_temp.


####  Accuracy Assessment - Model 

In [2]:
# use Exploratory Interpolation to compare Ordinary Kriging, Universal Kriging, and IDW
arcpy.ga.ExploratoryInterpolation(
    in_features="mn_temp_0320",
    value_field="max_tmpf",
    out_cv_table="ExploratoryInterpolation_temp",
    out_geostat_layer=None,
    interp_methods="ORDINARY_KRIGING;UNIVERSAL_KRIGING;IDW",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'D:\\spring2024\\GIS5572\\Lab3\\Lab3\\Lab3.gdb\\ExploratoryInterpolation_temp'>

In [4]:
# save the table into SDE
arcpy.conversion.TableToGeodatabase(
    Input_Table="ExploratoryInterpolation_temp",
    Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde"
)

<Result 'D:\\spring2024\\GIS5572\\Lab3\\Lab3\\PostgreSQL-34-gis5572(postgres).sde'>

#### Accuracy Assessment - Results

In [22]:
def differences_temp(true_points, interpolated_points):
    
    # get the name of interpolation method as part of the output point layer 
    interpolation = os.path.basename(interpolated_points)
    
    # spatal join
    output_join = f"diff_{interpolation}"
    arcpy.analysis.SpatialJoin(true_points, interpolated_points, output_join, "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "CLOSEST")

    # rename column
    arcpy.management.AlterField(output_join, "max_tmpf", "ground_truth", "ground_truth")
    arcpy.management.AlterField(output_join, "grid_code", "prediction", "prediction")

    # calculate difference
    fields = ["station", "ground_truth", "prediction", "difference"]
    arcpy.management.AddField(output_join, "difference", "DOUBLE")
    with arcpy.da.UpdateCursor(output_join, fields) as cursor:
        for row in cursor:
            row[3] = row[1] - row[2]  # "ground_truth" - "prediction"
            cursor.updateRow(row)
    arcpy.management.DeleteField(output_join,"Join_Count;TARGET_FID;date;min_tmpf;name;pointid","DELETE_FIELDS")

    print(f"Difference {interpolation} points layer created.")


In [24]:
true_points = "mn_temp_0320"

# ORDINARY_KRIGING
differences_temp(true_points, "Krig_ord_pts_temp")
# UNIVERSAL_KRIGING
differences_temp(true_points, "Krig_uni_pts_temp")
# IDW
differences_temp(true_points, "Idw_pts_temp")

Difference Krig_ord_pts_temp points layer created.
Difference Krig_uni_pts_temp points layer created.
Difference Idw_pts_temp points layer created.


#### store the accuracy assessment to PostGIS database

In [25]:
# store the accuracy assessment to PostGIS database
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="diff_Krig_ord_pts_temp;diff_Krig_uni_pts_temp;diff_Idw_pts_temp",
    Output_Geodatabase="\\PostgreSQL-34-gis5572(postgres).sde"
)


<Result 'D:\\spring2024\\GIS5572\\Lab3\\Lab3\\PostgreSQL-34-gis5572(postgres).sde'>

#### Adjust data for uploading to AGOL

In [62]:
# use BERNOULLI sampling method to reduce the points
cur.execute("CREATE TABLE sampled_idw_pts_temp AS SELECT * FROM idw_pts_temp TABLESAMPLE BERNOULLI(1)")

# commit changes to the database
conn.commit()

# close the cursor and connection
cur.close()
conn.close()

In [64]:
# check how many data after using BERNOULLI 
cur.execute("SELECT * FROM postgres.sampled_idw_pts_temp")

# fetch the results
results = cur.fetchall()

print(len(results))
# close the cursor and connection
cur.close()
conn.close()

813


In [68]:
# transfer coordinate to EPSG:4326
cur.execute("""
ALTER TABLE sampled_idw_pts_temp 
ALTER COLUMN shape TYPE geometry(Point, 4326) 
USING ST_Transform(shape, 4326);
""")
# Commit changes to the database
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()